# Problem 1: Portfolio Construction & Optimization

## Objective
Construct an optimal equity portfolio from Nifty 100 stocks that maximizes the Sharpe Ratio under realistic constraints.

## Method Overview
This notebook follows these steps:
1. Load historical price data
2. Compute returns and covariance matrix
3. Construct optimal portfolios (Markowitz & Risk Parity)
4. Compare performance with Nifty 50 benchmark


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [9]:
# Load IDX80 Excel (2025)
idx80_raw = pd.read_excel("3 Lamp Peng-00139-BEI POP - IDX80 - Jul 2025 Mayor.xlsx")

idx80_raw.head()


,nomor,Saham,Bobot,Pra Evaluasi,Pasca Evaluasi *\n(9% Cap),Keterangan\nTetap/Naik/\nTurun/Baru,Pra \nEvaluasi,Pasca Evaluasi,Keterangan\nTetap/Naik/\nTurun/Baru.1
0,1,AADI,0.2183,-,1699878471,Baru,-,0.007371,Baru
1,2,ACES,0.3999,6846443841,6846443841,Tetap,0.00203,0.001961,Tetap
2,3,ADMR,0.1253,5232938432,5122556137,Turun,0.004011,0.003792,Turun
3,4,ADRO,0.2854,9445986298,8778523248,Turun,0.011938,0.010716,Turun
4,5,AKRA,0.3255,6545960067,6533915982,Turun,0.00516,0.004975,Turun


In [17]:
stocks = idx80_raw["Saham"].dropna().unique()

len(stocks), stocks[:10]
tickers = [s + ".JK" for s in stocks]
tickers[:10]
start_date = "2021-01-01"
end_date   = "2025-01-01"

prices = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,
    progress=True
)["Adj Close"]

prices.head()



[*********************100%***********************]  80 of 80 completed


Ticker,AADI.JK,ACES.JK,ADMR.JK,ADRO.JK,AKRA.JK,AMMN.JK,AMRT.JK,ANTM.JK,ARTO.JK,ASII.JK,...,SMGR.JK,SMRA.JK,SRTG.JK,SSIA.JK,TAPG.JK,TKIM.JK,TLKM.JK,TOWR.JK,UNTR.JK,UNVR.JK
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,NaN,1386.035156,NaN,454.885590,472.359253,NaN,758.893494,1806.794312,3516.352295,4361.848633,...,10866.419922,754.607483,631.142700,568.276123,NaN,10474.500977,2687.039062,847.832520,15838.655273,6058.866699
2021-01-05,NaN,1434.954102,NaN,445.506561,466.581146,NaN,744.664185,1839.795044,3690.511230,4326.812988,...,10715.197266,759.153320,622.152039,558.478210,NaN,10499.030273,2671.640625,843.462219,15630.641602,5977.811035
2021-01-06,NaN,1410.494751,NaN,429.874695,459.358551,NaN,739.921082,1831.545044,3491.472412,4309.296387,...,10391.149414,727.332520,616.757690,548.680359,NaN,10351.846680,2594.647949,843.462219,15333.480469,5835.964355
2021-01-07,NaN,1451.260498,NaN,439.253845,492.582550,NaN,725.691833,2145.052734,3574.405273,4256.743164,...,10628.784180,727.332520,620.353882,563.377136,NaN,11357.595703,2610.046875,839.091980,15927.803711,5835.964355
2021-01-08,NaN,1467.566650,NaN,451.759308,489.693542,NaN,735.178040,2145.052734,3549.525391,4501.987793,...,10650.388672,736.424194,627.546448,568.276123,NaN,12657.709961,2748.633057,843.462219,16210.107422,5815.701172


In [ ]:
prices = prices.dropna(axis=1, thresh=int(0.9 * len(prices)))
prices.shape


In [18]:
prices.index = pd.to_datetime(prices.index)
def check_nan_by_year(prices, start_year, end_year):
    result = {}

    for year in range(start_year, end_year + 1):
        yearly_data = prices[prices.index.year == year]
        
        # saham yang MASIH ada NaN di tahun tersebut
        nan_stocks = yearly_data.columns[yearly_data.isna().any()].tolist()
        
        result[year] = nan_stocks

    return result
nan_by_year = check_nan_by_year(prices, 2020, 2025)
for year, stocks in nan_by_year.items():
    print(f"{year}: {len(stocks)} saham masih NaN")
    print(stocks)
    print("-" * 50)


2020: 0 saham masih NaN
[]
--------------------------------------------------
2021: 12 saham masih NaN
['AADI.JK', 'ADMR.JK', 'AMMN.JK', 'AVIA.JK', 'BUKA.JK', 'CMRY.JK', 'GOTO.JK', 'MBMA.JK', 'MTEL.JK', 'NCKL.JK', 'PGEO.JK', 'TAPG.JK']
--------------------------------------------------
2022: 6 saham masih NaN
['AADI.JK', 'AMMN.JK', 'GOTO.JK', 'MBMA.JK', 'NCKL.JK', 'PGEO.JK']
--------------------------------------------------
2023: 5 saham masih NaN
['AADI.JK', 'AMMN.JK', 'MBMA.JK', 'NCKL.JK', 'PGEO.JK']
--------------------------------------------------
2024: 1 saham masih NaN
['AADI.JK']
--------------------------------------------------
2025: 0 saham masih NaN
[]
--------------------------------------------------


In [19]:
drop_stocks_2021 = [
    'AADI.JK', 'ADMR.JK', 'AMMN.JK', 'AVIA.JK', 'BUKA.JK',
    'CMRY.JK', 'GOTO.JK', 'MBMA.JK', 'MTEL.JK', 'NCKL.JK',
    'PGEO.JK', 'TAPG.JK'
]
# ambil periode post-Covid
prices_post = prices.loc['2021-01-01':]

# buang saham yang sudah kita tentukan
prices_post = prices_post.drop(columns=drop_stocks_2021, errors='ignore')

prices_post.shape
prices_post = prices_post.dropna()
